In [30]:
#对这个练习，将再次处理手写数字数据集，这次使用反向传播的前馈神经网络。
#将通过反向传播算法实现神经网络成本函数和梯度计算的非正则化和正则化版本。 还将实现随机权重初始化和使用网络进行预测的方法。
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [35]:
data = loadmat('ex4data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [36]:
X = data['X']
y = data['y']

X.shape, y.shape#看下维度

((5000, 400), (5000, 1))

In [37]:
#需要对y标签进行一次one-hot 编码。 
#one-hot 编码将类标签n（k类）转换为长度为k的向量，其中索引n为“hot”（1），而其余为0。 
#Scikitlearn有一个内置的实用程序，我们可以使用这个。
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

In [38]:
y[0], y_onehot[0,:]

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

In [39]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [56]:
def forward_propagate(X, theta1, theta2):
    # INPUT：参数值theta，数据X
    # OUTPUT：当前参数值下前项传播结果
    # TODO：根据参数和输入的数据计算前项传播结果
    
    # STEP1：获取样本个数
    # your code here  (appro ~ 1 lines)
    m = X.shape[0]
   
    # STEP2：实现神经网络正向传播
    # your code here  (appro ~ 5 lines)    
    a1 = np.insert(X,0,values=np.ones(m),axis=1) #给X矩阵插入一行零元素
    z2 = a1*theta1.T
    a2 = np.insert(sigmoid(z2),0,values=np.ones(m),axis=1)  #注意插入零元素
    z3 = a2*theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [57]:
def cost(params, input_size, hidden_size, num_labels, X, y, lamda):
    # INPUT：神经网络参数，输入层维度，隐藏层维度，训练数据及标签，正则化参数
    # OUTPUT：当前参数值下的代价函数
    # TODO：根据上面的公式计算代价函数
    
    # STEP1：获取样本个数
    # your code here  (appro ~ 1 lines)
    m = X.shape[0]
    
    # STEP2：将矩阵X,y转换为numpy型矩阵
    # your code here  (appro ~ 2 lines)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # STEP3：从params中获取神经网络参数，并按照输入层维度和隐藏层维度重新定义参数的维度
    # your code here  (appro ~ 2 lines)
    theta1 =  np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # STEP4：调用前面写好的前项传播函数
    # your code here  (appro ~ 1 lines)
    a1, z2, a2, z3, h = forward_propagate(X,theta1,theta2)
    
    # STEP5：初始化代价函数
    # your code here  (appro ~ 1 lines)
    J = 0
    
    # STEP6：根据公式计算代价函数
    for i in range(m):  #遍历每个样本
        # your code here  (appro ~ 2 lines)
        first_term = np.multiply(-y[i,:],np.log(h[i,:]))
        second_term = np.multiply((1-y[i,:]),np.log(1-h[i,:]))
        J += np.sum(first_term - second_term)
    J = J / m
    # STEP7：计算代价函数的正则化部分
    # your code here  (appro ~ 1 lines)
    J += (float(lamda)/(2*m))*(np.sum(np.power(theta1[:,1:],2))+np.sum(np.power(theta2[:,1:],2)))
    
    return J

In [58]:
# 初始化设置
input_size = 400
hidden_size = 25
num_labels = 10
lamda = 1

# 随机初始化完整网络参数大小的参数数组
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# 将参数数组解开为每个层的参数矩阵
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((25, 401), (10, 26))

In [59]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [60]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, lamda)

6.997520673376277

In [61]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [76]:
def backprop(params, input_size, hidden_size, num_labels, X, y, lamda):
    # INPUT：神经网络参数，输入层维度，隐藏层维度，训练数据及标签，正则化参数
    # OUTPUT：当前参数值下的代价函数
    # TODO：根据上面的公式计算代价函数
    
    # STEP1：获取样本个数
    # your code here  (appro ~ 1 lines)
    m = X.shape[0]
    
    # STEP2：将矩阵X,y转换为numpy型矩阵
    # your code here  (appro ~ 2 lines)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # STEP3：从params中获取神经网络参数，并按照输入层维度和隐藏层维度重新定义参数的维度
    # your code here  (appro ~ 2 lines)
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # STEP4：调用前面写好的前项传播函数
    # your code here  (appro ~ 1 lines)
    a1, z2, a2, z3, h = forward_propagate(X,theta1,theta2)
    
    # STEP5：初始化
    # your code here  (appro ~ 3 lines)
    J = 0
    delta1 = np.zeros(theta1.shape)#(25,401)
    delta2 = np.zeros(theta2.shape)#(10,26)
    
    # STEP6：计算代价函数(调用函数)
    # your code here  (appro ~ 1 lines)
    for i in range(m):
        first_term=np.multiply(-y[i,:],np.log(h[i,:]))
        second_term=np.multiply((1-y[i,:]),np.log(1-h[i,:]))
        J +=np.sum(first_term-second_term)
    J = J / m
    
    # STEP7：实现反向传播（这里用到的公式请参考原版作业PDF的第5页）
    for t in range(m):  #遍历每个样本
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        # your code here  (appro ~ 5 lines)
        d3t = ht - yt
        z2t = np.insert(z2t,0,values=np.ones(1))#(1,26)
        d2t = np.multiply((theta2.T*d3t.T).T,sigmoid_gradient(z2t))
        delta1 = delta1+(d2t[:,1:]).T*a1t
        delta2 = delta2+d3t.T*a2t
    
    # STEP8：加入正则化
    # your code here  (appro ~ 1 lines)
    delta1[:,1:] = delta1[:,1:]+(theta1[:,1:]*lamda)/m
    delta2[:,1:] = delta2[:,1:]+(theta2[:,1:]*lamda)/m  
    
    # STEP9：将梯度矩阵转换为单个数组
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [77]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape

(6.992190853848363, (10285,))

In [85]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate), 
                method='TNC', jac=True, options={'maxfun': 25})
fmin

 message: Max. number of function evaluations reached
 success: False
  status: 3
     fun: 3.177510836808159
       x: [-6.811e-02 -4.508e-02 ... -1.364e-01 -3.482e-02]
     nit: 2
     jac: [ 8.062e+00 -9.017e-08 ... -4.195e+01 -8.729e+01]
    nfev: 26

In [82]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[6],
       [6],
       [6],
       ...,
       [9],
       [9],
       [9]], dtype=int64)

In [83]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 19.439999999999998%
